# Excited states solvers

In [11]:
from qiskit_nature.second_q.drivers import GaussianForcesDriver

# if you ran Gaussian elsewhere and already have the output file
#driver = GaussianForcesDriver(logfile="CO2_freq_B3LYP_631g.log")
driver = GaussianForcesDriver(logfile="CO2_out_fake.OUT")

# if you want to run the Gaussian job from Qiskit
# driver = GaussianForcesDriver(
#                 ['#p B3LYP/6-31g Freq=(Anharm) Int=Ultrafine SCF=VeryTight',
#                  '',
#                  'CO2 geometry optimization B3LYP/6-31g',
#                  '',
#                  '0 1',
#                  'C  -0.848629  2.067624  0.160992',
#                  'O   0.098816  2.655801 -0.159738',
#                  'O  -1.796073  1.479446  0.481721',
#                  '',
#                  ''

In [12]:
from qiskit_nature.second_q.problems import HarmonicBasis

basis = HarmonicBasis([2, 2, 2, 2])

In [ ]:
from qiskit_nature.second_q.problems import VibrationalStructureProblem
from qiskit_nature.second_q.hamiltonians import hamiltonian

vibrational_problem = driver.run(basis=basis)
vibrational_problem.hamiltonian.truncation_order = 2
main_op, aux_ops = vibrational_problem.second_q_ops()

In [10]:
from qiskit_nature.second_q.mappers import DirectMapper

qubit_mapper = DirectMapper()
qubit_op = qubit_mapper.map(main_op)
print(qubit_op)

SparsePauliOp(['IIIIIIII', 'IIIIIIIZ', 'IIIIIIZI', 'IIIIIZII', 'IIIIXXII', 'IIIIYYII', 'IIIIZIII', 'IIIZIIII', 'IIZIIIII', 'IZIIIIII', 'ZIIIIIII', 'IIIIIZIZ', 'IIIIXXIZ', 'IIIIYYIZ', 'IIIIZIIZ', 'IIIIIZZI', 'IIIIXXZI', 'IIIIYYZI', 'IIIIZIZI', 'IIIZIIIZ', 'IIIZIIZI', 'IIIZIZII', 'IIIZXXII', 'IIIZYYII', 'IIIZZIII', 'IIZIIIIZ', 'IIZIIIZI', 'IIZIIZII', 'IIZIXXII', 'IIZIYYII', 'IIZIZIII', 'IZIIIIIZ', 'IZIIIIZI', 'IZIIIZII', 'IZIIXXII', 'IZIIYYII', 'IZIIZIII', 'IZIZIIII', 'IZZIIIII', 'ZIIIIIIZ', 'ZIIIIIZI', 'ZIIIIZII', 'ZIIIXXII', 'ZIIIYYII', 'ZIIIZIII', 'ZIIZIIII', 'ZIZIIIII'],
              coeffs=[ 4.85420003e+03+0.j, -6.18564597e+02+0.j, -1.86053067e+03+0.j,
 -3.49485635e+02+0.j, -2.58640489e+01+0.j, -2.58640489e+01+0.j,
 -1.04971911e+03+0.j, -1.11855863e+02+0.j, -3.42575167e+02+0.j,
 -1.11855863e+02+0.j, -3.42575167e+02+0.j,  1.23563563e+00+0.j,
  2.20504355e+01+0.j,  2.20504355e+01+0.j,  3.70690688e+00+0.j,
  3.70690688e+00+0.j,  6.61513066e+01+0.j,  6.61513066e+01+0.j,
  1.11207206e+0

We will also be sticking to the Jordan-Wigner mapping. To learn more about the various mappers available in Qiskit Nature, check out the [Qubit Mappers tutorial](06_qubit_mappers.ipynb).

## The Solver

After these steps we need to define a solver. The solver is the algorithm through which the excited states are computed. 

Let's first start with a purely classical example: the `NumPyEigensolver`. This algorithm exactly diagonalizes the Hamiltonian. Although it scales badly, it can be used on small systems to check the results of the quantum algorithms. 
Here, we are only interested to look at eigenstates with a given number of particles. To compute only those states a filter function can be passed to the `NumPyEigensolver`. A default filter function is already implemented in Qiskit Nature which you can use for this purpose.

We also need to specify the number of eigenvalues to be computed by the `NumPyEigensolver`. For this particular system, we are interested in the ground and first three excited states, so we will set `k=4` (which defaults to 1 so be sure to set this, otherwise you will only obtain the ground state!).

The excitation energies can also be accessed with the [qEOM algorithm](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.2.043140). The EOM method finds the excitation energies (differences in energy between the ground state and all $n$th excited states) by solving the following pseudo-eigenvalue problem.

$$
\begin{pmatrix}
    \text{M} & \text{Q}\\ 
    \text{Q*} & \text{M*}
\end{pmatrix}
\begin{pmatrix}
    \text{X}_n\\ 
    \text{Y}_n
\end{pmatrix}
= E_{0n}
\begin{pmatrix}
    \text{V} & \text{W}\\ 
    -\text{W*} & -\text{V*}
\end{pmatrix}
\begin{pmatrix}
    \text{X}_n\\ 
    \text{Y}_n
\end{pmatrix}
$$

with 

$$
M_{\mu_{\alpha}\nu_{\beta}} = \langle0| [(\hat{\text{E}}_{\mu_{\alpha}}^{(\alpha)})^{\dagger},\hat{\text{H}}, \hat{\text{E}}_{\nu_{\beta}}^{(\beta)}]|0\rangle
$$
$$
Q_{\mu_{\alpha}\nu_{\beta}} = -\langle0| [(\hat{\text{E}}_{\mu_{\alpha}}^{(\alpha)})^{\dagger}, \hat{\text{H}}, (\hat{\text{E}}_{\nu_{\beta}}^{(\beta)})^{\dagger}]|0\rangle
$$
$$
V_{\mu_{\alpha}\nu_{\beta}} = \langle0| [(\hat{\text{E}}_{\mu_{\alpha}}^{(\alpha)})^{\dagger}, \hat{\text{E}}_{\nu_{\beta}}^{(\beta)}]|0\rangle
$$
$$
W_{\mu_{\alpha}\nu_{\beta}} = -\langle0| [(\hat{\text{E}}_{\mu_\alpha}^{(\alpha)})^{\dagger}, (\hat{\text{E}}_{\nu_{\beta}}^{(\beta)})^{\dagger}]|0\rangle
$$

Although the previous equation can be solved classically, each matrix element must be measured on the quantum computer with the corresponding ground state. 
To use the qEOM as a solver in Qiskit, we have to define a ground state calculation first, which will provide the required ground state information to the algorithm. With this the qEOM solver can be initialized like so:

In [6]:
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, QEOM, EvaluationRule
from qiskit_nature.second_q.circuit.library import UVCCSD, VSCF
import numpy as np
from qiskit import QuantumCircuit

#Set up the problem using the vibrational Hamiltonian
problem = vibrational_problem

ansatz = UVCCSD([2, 2, 2, 2],  qubit_mapper, initial_state=VSCF(
    [2,2,2,2],
    qubit_mapper,
))

vqe = VQE(Estimator(), ansatz, SLSQP())
estimator = Estimator()
# This first part sets the ground state solver
# see more about this part in the ground state calculation tutorial
solver = VQE(estimator, ansatz, SLSQP())
vqe.initial_point = np.zeros(ansatz.num_parameters)
gse = GroundStateEigensolver(qubit_mapper, solver)

# The qEOM algorithm is simply instantiated with the chosen ground state solver and Estimator primitive
qeom_excited_states_solver = QEOM(gse, estimator, "sd", EvaluationRule.DIAG)

In [7]:

qeom_results = qeom_excited_states_solver.solve(problem=vibrational_problem)
print(qeom_results)

=== GROUND STATE ===
 
* Vibrational ground state energy (cm^-1): 2432.107742683936
The number of occupied modals for each mode is: 
- Mode 0: 1.0
- Mode 1: 1.0
- Mode 2: 1.0
- Mode 3: 1.0
 
=== EXCITED STATES ===
 
*   1: Vibrational excited state energy (cm^-1): 2908.88674891047
The number of occupied modals for each mode is
- Mode 0: 1.0
- Mode 1: 1.0
- Mode 2: 1.0
- Mode 3: 1.0
 
*   2: Vibrational excited state energy (cm^-1): 2908.88674928233
The number of occupied modals for each mode is
- Mode 0: 0.999999943545
- Mode 1: 0.999999943545
- Mode 2: 0.999999943545
- Mode 3: 0.999999943545
 
*   3: Vibrational excited state energy (cm^-1): 3424.363038866002
The number of occupied modals for each mode is
- Mode 0: 0.999999950952
- Mode 1: 0.999999950952
- Mode 2: 0.999999950952
- Mode 3: 0.999999950952
 
*   4: Vibrational excited state energy (cm^-1): 3842.8545999697
The number of occupied modals for each mode is
- Mode 0: 0.999999999966
- Mode 1: 0.999999999966
- Mode 2: 0.99999999

In [8]:
import tutorial_magics

%qiskit_version_table
%qiskit_copyright